In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'

!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to /tmp/pip-install-8u8oby0c/pytorch-tabnet
  Running command git clone -q https://github.com/dreamquark-ai/tabnet.git /tmp/pip-install-8u8oby0c/pytorch-tabnet
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-tabnet: filename=pytorch_tabnet-3.1.1-cp37-none-any.whl size=39236 sha256=4130c9e3cd04d97c2e0d980c478e70f5e1cd5d2937b3e546a6f7517a46bbada2
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lauf9lp/wheels/a5/fe/e1/d7be493728d1fb7284583f5449d4cad80139ee994ef420f522
Successfully built pytorch-tabnet


In [1]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

import pandas as pd
import numpy as np
np.random.seed(0)

from tqdm.notebook import tqdm

import os

from matplotlib import pyplot as plt
%matplotlib inline

path = '/content/drive/MyDrive/credit/'

train = pd.read_csv(path+'train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv(path+'test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submission = pd.read_csv(path+'sample_submission.csv')

data=pd.concat([train, test], axis=0)
data.shape

# 나이 변환
def days_to_age(x):
    return (x*-1)/365
data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(days_to_age)

# 마이너스 값 변환
def minus(x):
    return x * -1
data['begin_month'] = data['begin_month'].apply(minus)

data.loc[(data.DAYS_EMPLOYED)>=0,'DAYS_EMPLOYED'] = 0

data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(days_to_age)

data.loc[data['child_num'] >= 2,'child_num']=2

data['income_total'] = data['income_total'].astype(object)
data['income_total'] = data['income_total']/10000 
print(data['income_total'].value_counts(bins=10,sort=False))
data['income_total'].plot(kind='hist',bins=50,density=True)

count, bin_dividers =np.histogram(data['income_total'], bins=7)
bin_names=[int(i) for i in range(7) ]
data['income_total']=pd.cut(x=data['income_total'], bins=bin_dividers, labels=bin_names, include_lowest=True)

data['gender'] = data['gender'].replace(['F','M'],[0,1])
print('gender :')
print(data['gender'].value_counts())
print('--------------')

print('Having a car or not : ')
data['car'] = data['car'].replace(['N','Y'],[0,1])
print(data['car'].value_counts())
print('--------------')

print('Having house reality or not: ')
data['reality'] = data['reality'].replace(['N','Y'],[0,1])
print(data['reality'].value_counts())
print('--------------')
      
print('Having a phone or not: ')
print(data['phone'].value_counts())
print('--------------')
      

print('Having a email or not: ')
print(data['email'].value_counts())
print('--------------')
      

print('Having a work phone or not: ')
print(data['work_phone'].value_counts())
print('--------------')

from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
data['income_type']=label_encoder.fit_transform(data['income_type'])
data['edu_type']=label_encoder.fit_transform(data['edu_type'])
data['family_type']=label_encoder.fit_transform(data['family_type'])
data['house_type']=label_encoder.fit_transform(data['house_type'])
data['income_total']=label_encoder.fit_transform(data['income_total'])
data['occyp_type']=label_encoder.fit_transform(data['occyp_type'])

data.info()

train=data[:len(data)-10000]
test=data[len(data)-10000:]

train_x=train.drop('credit', axis=1)
train_y=train[['credit']]
test_x=test.drop('credit', axis=1)

np.random.seed(42)
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(train.shape[0],))

train_indices = train_x[train.Set=="train"].index
valid_indices = train_x[train.Set=="valid"].index


nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in tqdm(train_x.columns):
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train_x[col].nunique())
        l_enc = LabelEncoder()
        train_x[col] = train_x[col].fillna("NaN")
        train_x[col] = l_enc.fit_transform(train_x[col].values)
        try:
            test_X[col] = test_x[col].fillna("NaN")
            test_x[col] = l_enc.transform(test_x[col].values)
        except:
            print(f"Column {col} does not exist in test set")
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        training_mean = train_x.loc[train_indices, col].mean()
        train.fillna(training_mean, inplace=True)
        test.fillna(training_mean, inplace=True)

features = [ col for col in train_x.columns] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

X_train = train_x[features].values[train_indices]
y_train = train_y.values[train_indices]

X_valid = train_x[features].values[valid_indices]
y_valid = train_y.values[valid_indices]

X_test = test_x[features].values

ModuleNotFoundError: No module named 'pytorch_tabnet'

In [ ]:
clf = TabNetMultiTaskClassifier(n_steps=1,
                                cat_idxs=cat_idxs,
                                cat_dims=cat_dims,
                                cat_emb_dim=1,
                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=2e-2),
                                scheduler_params={"step_size":50,
                                                  "gamma":0.9},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                mask_type='entmax', 
                                lambda_sparse=0, 
                       
                      )

Device used : cpu


In [ ]:
max_epochs = 1000
clf.fit(
    X_train=X_train, y_train=y_train,
    max_epochs=max_epochs ,
    patience=50, # please be patient ^^
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False,
)

No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.96714 |  0:00:01s
epoch 1  | loss: 0.8842  |  0:00:03s
epoch 2  | loss: 0.87925 |  0:00:04s
epoch 3  | loss: 0.87642 |  0:00:06s
epoch 4  | loss: 0.87373 |  0:00:08s
epoch 5  | loss: 0.87046 |  0:00:09s
epoch 6  | loss: 0.86463 |  0:00:11s
epoch 7  | loss: 0.83399 |  0:00:12s
epoch 8  | loss: 0.80745 |  0:00:14s
epoch 9  | loss: 0.80176 |  0:00:15s
epoch 10 | loss: 0.80014 |  0:00:17s
epoch 11 | loss: 0.7981  |  0:00:18s
epoch 12 | loss: 0.79593 |  0:00:20s
epoch 13 | loss: 0.79232 |  0:00:22s
epoch 14 | loss: 0.79391 |  0:00:23s
epoch 15 | loss: 0.79033 |  0:00:25s
epoch 16 | loss: 0.78848 |  0:00:26s
epoch 17 | loss: 0.78808 |  0:00:28s
epoch 18 | loss: 0.78478 |  0:00:29s
epoch 19 | loss: 0.78071 |  0:00:31s
epoch 20 | loss: 0.77975 |  0:00:33s
epoch 21 | loss: 0.77881 |  0:00:34s
epoch 22 | loss: 0.77882 |  0:00:36s
epoch 23 | loss: 0.77794 |  0:00:37s
epoch 24 | loss: 0.77438 |  0:00:39s
ep

In [ ]:
preds_valid = clf.predict_proba(X_valid) 

In [ ]:
preds = clf.predict_proba(X_test)

In [ ]:
clf.feature_importances_

array([0.00619394, 0.00576959, 0.08090499, 0.04623143, 0.02449365,
       0.03460251, 0.13866297, 0.01080321, 0.03821061, 0.06909439,
       0.06285745, 0.02037808, 0.04075837, 0.00378588, 0.21078522,
       0.12872638, 0.0132634 , 0.06447791])

In [ ]:
i, j, k = [], [], []
for row in range(10000):
  i.append(preds[0][row][0])
  j.append(preds[0][row][1])
  k.append(preds[0][row][2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [ ]:
submission['0'] = i
submission['1'] = j
submission['2'] = k
submission.head(10)

,index,0,1,2
0,26457,0.110453,0.352578,0.536969
1,26458,0.253677,0.411024,0.335299
2,26459,0.110948,0.571531,0.317522
3,26460,0.021644,0.179785,0.798571
4,26461,0.009317,0.000071,0.990611
5,26462,0.054142,0.061007,0.884851
6,26463,0.190424,0.782257,0.027318
7,26464,0.017569,0.856357,0.126074
8,26465,0.247234,0.188375,0.564391
9,26466,0.000044,0.013179,0.986777


In [ ]:
submission.to_csv(path+"submisison_tabnet.csv",index=False)

In [ ]:
prediction = pd.read_csv(path+'submisison_tabnet.csv')

In [ ]:
prediction

,index,0,1,2
0,26457,0.110453,0.352578,0.536969
1,26458,0.253677,0.411024,0.335299
2,26459,0.110948,0.571531,0.317522
3,26460,0.021644,0.179785,0.798571
4,26461,0.009317,0.000071,0.990611
...,...,...,...,...
9995,36452,0.206809,0.302276,0.490915
9996,36453,0.021563,0.103430,0.875007
9997,36454,0.000578,0.024714,0.974708
9998,36455,0.023066,0.004329,0.972605
